In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from tensorflow.keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [16]:
true = pd.read_csv("../input/fake-and-real-news-dataset/True.csv")
false = pd.read_csv("../input/fake-and-real-news-dataset/Fake.csv")

In [17]:
true.head()

In [18]:
false.head()

In [19]:
true.shape, false.shape

In [20]:
true['category'] = 1
false['category'] = 0

In [21]:
true.head()

In [22]:
df = pd.concat([true, false], axis=0)

In [23]:
df

In [24]:
sns.set_style('darkgrid')
sns.countplot(df.category)

#### **This dataset is balanced**

In [26]:
df.isnull().sum()

In [27]:
df.title.count()

In [28]:
df.subject.value_counts()

In [29]:
plt.figure(figsize=(12, 8))
sns.set(style="whitegrid", font_scale=1.2)
chart = sns.countplot(x='subject', hue='category', data=df)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

In [30]:
df['text'] = df['text'] + " " + df['title']
del df['title'], df['subject'], df['date']

In [31]:
df.head()

In [32]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

### **Data Cleaning**

In [36]:
def strip_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Removing URL's
def remove_url(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_url(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    
    return text

df['text'] = df['text'].apply(denoise_text)

In [38]:
df.head()

#### **WordCloud for real text (category = 1)**

In [39]:
plt.figure(figsize=(20, 20))
wc = WordCloud(max_words=2000, width=1600, height=800, stopwords=STOPWORDS).generate(" ".join(df[df.category == 1].text))
plt.imshow(wc, interpolation='bilinear')

#### **WordCloud for fake text (category = 0)**

In [40]:
plt.figure(figsize=(20, 20))
wc = WordCloud(max_words=2000, width=1600, height=800, stopwords=STOPWORDS).generate(" ".join(df[df.category == 0].text))
plt.imshow(wc, interpolation='bilinear')

#### **Number of Characters in texts**

In [42]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
text_len = df[df['category']==1]['text'].str.len()
ax1.hist(text_len, color='red')
ax1.set_title('Real text')
text_len = df[df['category']==0]['text'].str.len()
ax2.hist(text_len, color='green')
ax2.set_title('Fake text')
fig.suptitle("Number of characters in texts")
plt.show()

#### **Number of words in each text**

In [43]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
text_len = df[df['category']==1]['text'].str.split().map(lambda x: len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Real text')
text_len = df[df['category']==0]['text'].str.split().map(lambda x: len(x))
ax2.hist(text_len, color='green')
ax2.set_title('Fake text')
fig.suptitle("Number of words in each text")
plt.show()

In [52]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
word = df[df['category'] == 1]['text'].str.split().apply(lambda x: [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)), ax=ax1, color='red')
ax1.set_title('Read text')
word = df[df['category'] == 0]['text'].str.split().apply(lambda x: [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)), ax=ax2, color='green')
ax2.set_title('Fake text')
fig.suptitle('Average word length in each text')
plt.show()

In [56]:
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(df.text)

In [57]:
corpus[:10]

In [59]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = dict(most_common)
most_common

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

#### **Unigram Analysis**

In [62]:
plt.figure(figsize=(20,10))
most_common_uni = get_top_text_ngrams(df.text, 10, 1)
most_common_uni = dict(most_common_uni)
sns.barplot(x=list(most_common_uni.values()), y=list(most_common_uni.keys()))

#### **Bigram Analysis**

In [63]:
plt.figure(figsize=(20,10))
most_common_uni = get_top_text_ngrams(df.text, 10, 2)
most_common_uni = dict(most_common_uni)
sns.barplot(x=list(most_common_uni.values()), y=list(most_common_uni.keys()))

#### **Trigram Analysis**

In [65]:
plt.figure(figsize=(20,10))
most_common_uni = get_top_text_ngrams(df.text, 10, 3)
most_common_uni = dict(most_common_uni)
sns.barplot(x=list(most_common_uni.values()), y=list(most_common_uni.keys()))

In [66]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.category, random_state=0)

In [67]:
X_train

In [68]:
max_features = 10000
max_len = 300

In [71]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences=tokenized_train, maxlen=max_len)

In [108]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=max_len)

In [74]:
X_train[0]

In [75]:
EMBEDDING_FILE = '../input/glovetwitter27b100dtxt/glove.twitter.27B.100d.txt'

In [76]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [94]:
all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(loc=emb_mean, scale=emb_std, size=(nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [111]:
batch_size = 256
epochs = 15
embed_size = 100

In [112]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

In [113]:
from tensorflow import keras
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=max_len, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [114]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])